## Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [21]:
import pandas as pd

In [22]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [25]:
statist = ratings.groupby('userId').agg(['max', 'min', 'count'])
statist = statist.loc[statist['timestamp']['count'] > 100]
statist.head()

movieId           rating              timestamp                  
           max min count    max  min count         max         min count
userId                                                                  
4         4006  10   204    5.0  1.0   204   949982274   949778714   204
8        44004  32   116    5.0  0.5   116  1154474527  1154389340   116
15      161155   1  1700    5.0  0.5  1700  1469330735   997937239  1700
17       34437   6   363    5.0  0.5   363  1127476640  1127468587   363
19        4970   1   423    5.0  1.0   423   855195373   855190091   423

In [26]:
statist['timelife'] = statist['timestamp']['max'] - statist['timestamp']['min']
statist.loc[:, ['userId', 'timelife']]

,timelife
,
userId,
4,203560
8,85187
15,471393496
17,8053
19,5282
...,...
656,3053
659,31609411


## Задание 2

Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [27]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [28]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [29]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [30]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [47]:
joined_one = rzd.merge(auto, how='outer').merge(air, how='outer')
joined_one.fillna(0, inplace=True) #мне кажется достаточно логичным для такой сводной таблицы заменить NaN на 0
joined_one

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [48]:
joined_two = joined_one.merge(client_base, how='outer')
joined_two

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4


##  Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

ОТВЕТ:

Для описания точного местонахождения точки в мире используются широта и долгота, каждая состоит из 5-6 цифр (в случае широты) и 5-7 цифр (в случае долготы) а также указания аббревиатуру, указывающую, в какой половине лежит точка (северная или южная широта и восточная или западная долгота). Так как неудобно в одном столбце вводить и числа и буквы, то можно принять северную широту за положительное направление исчесление градусов, а южное, соответственно, за отрицательное. Аналогично для долготы: восточная - положительное направление (от 0 градусов до 180), а западное - за отрицательное (от 0 до -180 градусов). Для этого можно написать несложную функцию, которая в зависимости от аббревиатуры приписывает числу знак и возвращает координаты только в формате float. Итого получится 2 столбца: широта и долгота. Также можно указывать город в соответствии с координатами. Это позволит разделить пользователей на группы по городу проживания.

Если у одного и того же пользователя могут быть значительно удаленные друг от друга координаты, то можно придумать несколько вариантов оформления данных. Например, если для каждого пользователя известны координаты при каждом использовании приложения и еще какой-то опознавательный знак, скажем, дата, когда пользователь с заданными координатами заходил в приложение, то можно допустить дубликаты по столбцу user_id, и для каждого посещения приложения указывать отдельную статистику. Но это может привести к существенному увеличению количества данных в случае, если изначально не предполагалось вести учет каждого посещения. Кроме того, пользователь попадет сразу в несколько групп пользователей, разделенных по городам.
Либо можно для каждого пользователя в каждый столбец широта/долгота записывать список из координат в случае достаточно сильной удаленности их друг от друга (также можно написать функцию, задав в ней порог удаленности новой координаты от всех предыдущих, в случае превышения которого новая координата вносится в список). Выбор стратегии определяется исходя из особенностей приложения, его работы и целей аналитики.

Кроме выявляения города можно также отметить насколько часто и далеко перемещается пользователь. Также можно определить в каком районе чаще всего совершаются покупки.